In [1]:
from pyspark.sql import SparkSession,Row
from pyspark.sql.functions import collect_list,flatten
from pyspark import SparkContext
from collections import Counter
from graphframes import GraphFrame

In [2]:
sc=SparkContext()
spark = SparkSession.builder.appName("Tweeter_project").master("local[4]").getOrCreate()
#spark.sparkContext.setLogLevel('ERROR')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/04 08:43:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#tweetsdf_initial = spark.read.format('json').option("header", "true").option("inferSchema","true").load("/Users/mohamed.soueilim@airliquide.com/Documents/projettweet/DatasetTweets_elections_US/DataSetUsElections03112020/datasethistoriquePassedElectionsUS_20201110-175813.jsonl")


tweetsdf_initial=spark.read.json("/Users/mohamed.soueilim@airliquide.com/Documents/projettweet/DatasetTweets_elections_US/DataSetUsElections03112020/datasethistoriquePassedElectionsUS_20201110-175813.jsonl")

23/07/04 08:43:57 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
23/07/04 08:44:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
tweetsdf=tweetsdf_initial.select("user.id_str","entities.hashtags.text","entities.user_mentions.screen_name").withColumnRenamed("screen_name","mentions")\
                                                                                                        .withColumnRenamed("id_str","user_id")\
                                                                                                        .withColumnRenamed("text","hashtags")


In [5]:
tweetsdf.show()

+-------------------+--------------------+--------------------+
|            user_id|            hashtags|            mentions|
+-------------------+--------------------+--------------------+
|          329325254|          [BREAKING]|           [gathara]|
|         3044562479|                  []|   [mehmetcilingiro]|
|         3500701458|                  []|        [BabaGlocal]|
|          100822546|                  []|          [JoeBiden]|
|          872796554|                  []|          [JoeBiden]|
|1212941673174458369|         [Periscope]|[GovRidge, RepsFo...|
|           34570323|                  []|                  []|
|1189648405678608384|                  []|[TeaPainUSA, JoeB...|
| 920027823846371344|                  []|[dmx_biographer, ...|
|1175776815018926082|       [ElectionDay]|                  []|
|         4535283420|                  []|        [NYinLA2121]|
|          291678033|                  []|                  []|
|1227714870616834048|[PleaseSaveTheseC..

In [6]:

tweetsdf=tweetsdf.groupBy("user_id").agg(flatten(collect_list(tweetsdf.hashtags)).alias("hashtags"),flatten(collect_list(tweetsdf.mentions)).alias("mentions"))


In [7]:
verticesdf=tweetsdf.select("user_id").withColumnRenamed("user_id","id")
tweetsdf = tweetsdf.limit(300)
tweetsdf.show()

+-------------------+--------------------+--------------------+
|            user_id|            hashtags|            mentions|
+-------------------+--------------------+--------------------+
|1000025521445629953|   [USElections2020]|[OkikeDan, realDo...|
|1000060779100495872|                  []|      [Weedledouble]|
|1000121542540394499|                  []|            [spi916]|
|1000144920529817600|                  []|[wobbulz, TheGrav...|
|1000174557439897600|        [VoteHimOut]|[votevets, SenDuc...|
|         1000182746|                  []|           [atrupar]|
|         1000204363|                  []|          [JoeBiden]|
|1000467976922025987|                  []|                  []|
|1000572465729355776|         [Trump2020]|   [mobarez_nastooh]|
|         1000657808|                  []|      [sisterquiche]|
|         1000692744|                  []|       [nowthisnews]|
|1000734463637139456|                  []|       [nowthisnews]|
|1000760316806901762|[GullibleWhiteMal..

In [9]:
tweetsRdd=tweetsdf.rdd

print("1 RDD Print\n")
print(tweetsRdd.collect())

1 RDD Print

[Row(user_id='1000025521445629953', hashtags=['USElections2020'], mentions=['OkikeDan', 'realDonaldTrump']), Row(user_id='1000060779100495872', hashtags=[], mentions=['Weedledouble']), Row(user_id='1000121542540394499', hashtags=[], mentions=['spi916']), Row(user_id='1000144920529817600', hashtags=[], mentions=['wobbulz', 'TheGraviter', 'Ivan_Is_Back', 'JoeBiden']), Row(user_id='1000174557439897600', hashtags=['VoteHimOut'], mentions=['votevets', 'SenDuckworth']), Row(user_id='1000182746', hashtags=[], mentions=['atrupar']), Row(user_id='1000204363', hashtags=[], mentions=['JoeBiden']), Row(user_id='1000467976922025987', hashtags=[], mentions=[]), Row(user_id='1000572465729355776', hashtags=['Trump2020'], mentions=['mobarez_nastooh']), Row(user_id='1000657808', hashtags=[], mentions=['sisterquiche']), Row(user_id='1000692744', hashtags=[], mentions=['nowthisnews']), Row(user_id='1000734463637139456', hashtags=[], mentions=['nowthisnews']), Row(user_id='1000760316806901762'

In [10]:
tweetsRdd=tweetsRdd.map(lambda user: (user.user_id,{"hashtags":dict(Counter(user.hashtags)),"mentions":dict(Counter(user.mentions))}))

print("2 RDD Print\n")
print(tweetsRdd.collect())

2 RDD Print

[('1000025521445629953', {'hashtags': {'USElections2020': 1}, 'mentions': {'OkikeDan': 1, 'realDonaldTrump': 1}}), ('1000060779100495872', {'hashtags': {}, 'mentions': {'Weedledouble': 1}}), ('1000121542540394499', {'hashtags': {}, 'mentions': {'spi916': 1}}), ('1000144920529817600', {'hashtags': {}, 'mentions': {'wobbulz': 1, 'TheGraviter': 1, 'Ivan_Is_Back': 1, 'JoeBiden': 1}}), ('1000174557439897600', {'hashtags': {'VoteHimOut': 1}, 'mentions': {'votevets': 1, 'SenDuckworth': 1}}), ('1000182746', {'hashtags': {}, 'mentions': {'atrupar': 1}}), ('1000204363', {'hashtags': {}, 'mentions': {'JoeBiden': 1}}), ('1000467976922025987', {'hashtags': {}, 'mentions': {}}), ('1000572465729355776', {'hashtags': {'Trump2020': 1}, 'mentions': {'mobarez_nastooh': 1}}), ('1000657808', {'hashtags': {}, 'mentions': {'sisterquiche': 1}}), ('1000692744', {'hashtags': {}, 'mentions': {'nowthisnews': 1}}), ('1000734463637139456', {'hashtags': {}, 'mentions': {'nowthisnews': 1}}), ('1000760316

In [11]:
def func1(user):
    finalList=[]
    hashtags_number=zip(user[1]['hashtags'].keys(),user[1]['hashtags'].values())
    for i in hashtags_number:
        a=list(i)
        a[0]=("hashtag",a[0])
        a[1]=(a[1],user[0])
        finalList.append(a)

    mentions_number=zip(user[1]['mentions'].keys(),user[1]['mentions'].values())
    for i in mentions_number:
        a=list(i)
        a[0]=("mention",a[0])
        a[1]=(a[1],user[0])
        finalList.append(a)

    return finalList

In [12]:
tweetsRdd=tweetsRdd.flatMap(func1)

In [13]:
print("3 RDD Print\n")
print(tweetsRdd.collect())

3 RDD Print

[[('hashtag', 'USElections2020'), (1, '1000025521445629953')], [('mention', 'OkikeDan'), (1, '1000025521445629953')], [('mention', 'realDonaldTrump'), (1, '1000025521445629953')], [('mention', 'Weedledouble'), (1, '1000060779100495872')], [('mention', 'spi916'), (1, '1000121542540394499')], [('mention', 'wobbulz'), (1, '1000144920529817600')], [('mention', 'TheGraviter'), (1, '1000144920529817600')], [('mention', 'Ivan_Is_Back'), (1, '1000144920529817600')], [('mention', 'JoeBiden'), (1, '1000144920529817600')], [('hashtag', 'VoteHimOut'), (1, '1000174557439897600')], [('mention', 'votevets'), (1, '1000174557439897600')], [('mention', 'SenDuckworth'), (1, '1000174557439897600')], [('mention', 'atrupar'), (1, '1000182746')], [('mention', 'JoeBiden'), (1, '1000204363')], [('hashtag', 'Trump2020'), (1, '1000572465729355776')], [('mention', 'mobarez_nastooh'), (1, '1000572465729355776')], [('mention', 'sisterquiche'), (1, '1000657808')], [('mention', 'nowthisnews'), (1, '10006

In [14]:
def func2(a,b):
    if type(a).__name__=="tuple":
        return [a]+[b]
    else:
        return a+[b]

In [15]:
tweetsRdd=tweetsRdd.reduceByKey(func2)

In [16]:
print("4 RDD Print\n")
print(tweetsRdd.collect())

4 RDD Print

[(('hashtag', 'USElections2020'), (1, '1000025521445629953')), (('mention', 'OkikeDan'), (1, '1000025521445629953')), (('mention', 'realDonaldTrump'), [(1, '1000025521445629953'), (1, '1000958571067080705'), (6, '1004449639'), (1, '1009945881016193024'), (1, '101162534'), (1, '1011985857916686338'), (2, '1011994430688059392'), (1, '1012293325'), (1, '101281667'), (1, '1013336133202206720'), (2, '1014870379960193025')]), (('mention', 'Weedledouble'), (1, '1000060779100495872')), (('mention', 'spi916'), [(1, '1000121542540394499'), (1, '1012385300788797441')]), (('mention', 'wobbulz'), (1, '1000144920529817600')), (('mention', 'TheGraviter'), (1, '1000144920529817600')), (('mention', 'Ivan_Is_Back'), (1, '1000144920529817600')), (('mention', 'JoeBiden'), [(1, '1000144920529817600'), (1, '1000204363'), (1, '1000760316806901762'), (2, '1000958571067080705'), (1, '1001374315'), (4, '1001467780670017537'), (1, '1001589257973182465'), (7, '1001808180702318592'), (1, '100218841369

In [17]:
def func3(a):
    a=a[1]
    if type(a).__name__=="tuple":
        return []
    else:
        result=[]
        for i in range(len(a)):
            for j in range(i+1,len(a)):
                result.append(((a[i][1],a[j][1]),min(a[i][0],a[j][0])))

        return result

In [18]:

tweetsRdd=tweetsRdd.flatMap(func3)

In [19]:

print("5 RDD Print\n")
print(tweetsRdd.collect())

5 RDD Print

[(('1000025521445629953', '1000958571067080705'), 1), (('1000025521445629953', '1004449639'), 1), (('1000025521445629953', '1009945881016193024'), 1), (('1000025521445629953', '101162534'), 1), (('1000025521445629953', '1011985857916686338'), 1), (('1000025521445629953', '1011994430688059392'), 1), (('1000025521445629953', '1012293325'), 1), (('1000025521445629953', '101281667'), 1), (('1000025521445629953', '1013336133202206720'), 1), (('1000025521445629953', '1014870379960193025'), 1), (('1000958571067080705', '1004449639'), 1), (('1000958571067080705', '1009945881016193024'), 1), (('1000958571067080705', '101162534'), 1), (('1000958571067080705', '1011985857916686338'), 1), (('1000958571067080705', '1011994430688059392'), 1), (('1000958571067080705', '1012293325'), 1), (('1000958571067080705', '101281667'), 1), (('1000958571067080705', '1013336133202206720'), 1), (('1000958571067080705', '1014870379960193025'), 1), (('1004449639', '1009945881016193024'), 1), (('10044496

In [20]:
tweetsRdd=tweetsRdd.reduceByKey(lambda count1,count2: count1+count2)

In [21]:
print("6 RDD Print\n")
print(tweetsRdd.collect())

6 RDD Print

[(('1000025521445629953', '1000958571067080705'), 1), (('1000025521445629953', '1004449639'), 1), (('1000025521445629953', '1009945881016193024'), 1), (('1000025521445629953', '101162534'), 1), (('1000025521445629953', '1011985857916686338'), 1), (('1000025521445629953', '1011994430688059392'), 1), (('1000025521445629953', '1012293325'), 1), (('1000025521445629953', '101281667'), 1), (('1000025521445629953', '1013336133202206720'), 1), (('1000025521445629953', '1014870379960193025'), 1), (('1000958571067080705', '1004449639'), 2), (('1000958571067080705', '1009945881016193024'), 1), (('1000958571067080705', '101162534'), 1), (('1000958571067080705', '1011985857916686338'), 1), (('1000958571067080705', '1011994430688059392'), 1), (('1000958571067080705', '1012293325'), 1), (('1000958571067080705', '101281667'), 1), (('1000958571067080705', '1013336133202206720'), 1), (('1000958571067080705', '1014870379960193025'), 1), (('1004449639', '1009945881016193024'), 1), (('10044496

In [22]:
def func4(a):
    return Row(src=a[0][0], dst=a[0][1], weight=a[1])

In [23]:
def func4(a):
    return Row(src=a[0][0], dst=a[0][1], weight=a[1])

In [24]:
print("7 RDD Print\n")
print(tweetsRdd.collect())

7 RDD Print

[(('1000025521445629953', '1000958571067080705'), 1), (('1000025521445629953', '1004449639'), 1), (('1000025521445629953', '1009945881016193024'), 1), (('1000025521445629953', '101162534'), 1), (('1000025521445629953', '1011985857916686338'), 1), (('1000025521445629953', '1011994430688059392'), 1), (('1000025521445629953', '1012293325'), 1), (('1000025521445629953', '101281667'), 1), (('1000025521445629953', '1013336133202206720'), 1), (('1000025521445629953', '1014870379960193025'), 1), (('1000958571067080705', '1004449639'), 2), (('1000958571067080705', '1009945881016193024'), 1), (('1000958571067080705', '101162534'), 1), (('1000958571067080705', '1011985857916686338'), 1), (('1000958571067080705', '1011994430688059392'), 1), (('1000958571067080705', '1012293325'), 1), (('1000958571067080705', '101281667'), 1), (('1000958571067080705', '1013336133202206720'), 1), (('1000958571067080705', '1014870379960193025'), 1), (('1004449639', '1009945881016193024'), 1), (('10044496

In [25]:
edgesdf = tweetsRdd.toDF()

In [26]:
print("8 DF Print\n")
edgesdf.show()

8 DF Print

+--------------------+---+
|                  _1| _2|
+--------------------+---+
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000025521445629...|  1|
|{1000958571067080...|  2|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1000958571067080...|  1|
|{1004449639, 1009...|  1|
+--------------------+---+
only showing top 20 rows



In [27]:
verticesdf.show()

+-------------------+
|                 id|
+-------------------+
|1126329912774709248|
|          376818440|
|1260816796539514880|
|          525936964|
|1131773761353596928|
|           65405641|
|          540397056|
| 800540649808281600|
|          262556004|
|1293049649838792704|
|          104617158|
|         1696937120|
|          257836766|
|           16532233|
|          623087635|
|1132297945343246338|
|         2549171851|
|         1387267770|
|         3295441437|
|           97378085|
+-------------------+
only showing top 20 rows



In [ ]:
from graphframes import GraphFrame


graph = GraphFrame(verticesdf, edgesdf)

etiquettes_assignees = graph.labelPropagation(maxIter=5)

etiquettes_assignees.show()

etiquettes_assignees.write.csv("hdfs://localhost:9000/communautés")